In [1]:
!pip install transformers
!pip install --upgrade torch torchvision torchaudio

In [2]:
! pip install wandb

In [3]:
!pip install scikit-learn

In [4]:
!pip install numpy

In [5]:
import json
import torch
import random
import numpy as np
import torch.nn as nn
import wandb
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [6]:
from sklearn.model_selection import train_test_split

# Load the data from train.json
with open('/kaggle/input/traintest/train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Split the data into train and test sets (85% train, 15% test)
train_data, test_data = train_test_split(data, test_size=0.15, random_state=42)

# Save the train and test data to separate files
with open('train_split.json', 'w', encoding='utf-8') as train_file:
    json.dump(train_data, train_file, ensure_ascii=False, indent=4)

with open('test_split.json', 'w', encoding='utf-8') as test_file:
    json.dump(test_data, test_file, ensure_ascii=False, indent=4)


In [7]:
# Đọc dữ liệu từ tập tin train_split.json vào biến train_data
with open('train_split.json', 'r', encoding='utf-8') as train_file:
    train_data = json.load(train_file)

# Đọc dữ liệu từ tập tin test_split.json vào biến test_data
with open('test_split.json', 'r', encoding='utf-8') as test_file:
    test_data = json.load(test_file)

In [8]:
def preprocess_data(data):
    input_ids = []
    attention_masks = []
    labels = []
    max_length=512
    for sample in data:
        encoded_dict = tokenizer.encode_plus(
                            sample['question'],
                            sample['text'],
                            add_special_tokens=True,
                            truncation=True,
                            max_length=max_length,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt'
                       )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        labels.append(1 if sample['label'] else 0)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels


In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
train_input_ids, train_attention_masks, train_labels = preprocess_data(train_data)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [11]:
test_input_ids, test_attention_masks, test_labels = preprocess_data(test_data)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [12]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [13]:
wandb.init(project='zaloqa-bert', entity='hdghung2912')

wandb: Currently logged in as: hdghung2912. Use `wandb login --relogin` to force relogin


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [15]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

In [16]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [17]:
num_epochs = 3
for epoch in range(num_epochs):
  total_loss = 0.0
  total_correct = 0
  total_samples = 0
  progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)

  for batch in progress_bar:
      batch_input_ids, batch_attention_masks, batch_labels = batch
      batch_input_ids, batch_attention_masks, batch_labels = batch_input_ids.to(device), batch_attention_masks.to(device), batch_labels.to(device)

      optimizer.zero_grad()
      outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
      loss = outputs.loss
      logits = outputs.logits
      _, predicted = torch.max(logits, 1)
      total_correct += torch.sum(predicted == batch_labels).item()
      total_samples += batch_labels.size(0)

      loss.backward()
      optimizer.step()

      total_loss += loss.item()

      progress_bar.set_postfix({'Loss': total_loss / len(train_dataloader)})  # Hiển thị loss trung bình

  # Tính và log accuracy
  train_accuracy = total_correct / total_samples
  wandb.log({"train_loss": total_loss / len(train_dataloader), "train_accuracy": train_accuracy}, step=epoch)

  print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {total_loss / len(train_dataloader):.4f}, Accuracy: {train_accuracy:.4f}")

# Đánh dấu kết thúc quá trình huấn luyện
wandb.finish()

Epoch 1/3, Average Loss: 0.4732, Accuracy: 0.7789


Epoch 2/3, Average Loss: 0.2908, Accuracy: 0.8820


Epoch 3/3, Average Loss: 0.1437, Accuracy: 0.9462


train_accuracy,▁▅█
train_loss,█▄▁
train_accuracy,0.9462
train_loss,0.14366


In [18]:

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        batch_input_ids, batch_attention_masks, batch_labels = batch
        batch_input_ids, batch_attention_masks = batch_input_ids.to(device), batch_attention_masks.to(device)

        outputs = model(batch_input_ids, attention_mask=batch_attention_masks)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=1)

        predictions.extend(batch_predictions.cpu().numpy())
        true_labels.extend(batch_labels.cpu().numpy())

precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


Precision: 0.7909090909090909
Recall: 0.6904761904761905
F1-Score: 0.7372881355932204
Confusion Matrix:
[[1674  161]
 [ 273  609]]
